# Basic LLM usage

Draive framework provides various ways to use LLM depending on the use case. The simplest interface is to generate text by using `generate_text` function. We can use it to make a simple text completion function.

In [1]:
from draive import generate_text


async def text_completion(text: str) -> str:
    # generate text is a simple interface for generating text
    return await generate_text(
        # We have to provide instructions / system prompt to instruct the model
        instruction="Prepare the simplest completion of a given text",
        # Input is provided separately and can be easily parametrized
        input=text,
    )

The result of this function is a completion from a currently used model. What is a currently used model? We have to define it yet by providing basic setup of state and dependencies. In this example we are going to use OpenAI client, you have to provide the api key to that service in .env file with `OPENAI_API_KEY` key before running.

In [2]:
from draive import load_env

load_env()  # load .env

When we have .env we can prepare a context scope with OpenAI client and use our function. Lowest level interface is called LMM since draive supports multi-model solutions out of the box. Assigning `invocation` to `openai_lmm_invocation` and using it as the context scope state selects this provider for our completions. We have also define OpenAI client dependency which defines functions for accessing OpenAI services.

In [3]:
from draive import LMM, OpenAIClient, ctx, openai_lmm_invocation

async with ctx.new(  # prepare new context
    "text_completion",
    state=[LMM(invocation=openai_lmm_invocation)],  # set LMM to OpenAI
    dependencies=[OpenAIClient],  # use OpenAIClient dependency for accessing OpenAI services
):
    result: str = await text_completion(
        text="Roses are red...",
    )

    print(result)

Violets are blue


As we now know how to setup OpenAI as out LLM provider. We can start customizing it more by providing GPT model configuration.

In [4]:
from draive import OpenAIChatConfig

async with ctx.new(  # prepare the context
    "text_completion",
    state=[
        LMM(invocation=openai_lmm_invocation),
        OpenAIChatConfig(  # define GPT model configuration
            model="gpt-3.5-turbo",
            temperature=0.4,
        ),
    ],
    dependencies=[OpenAIClient],
):
    # now we are using gpt-3.5-turbo with temperature of 0.4
    result: str = await generate_text(
        instruction="Prepare simplest completion of given text",
        input="Roses are red...",
    )

    print("RESULT GPT 3.5 | temperature 0.4:", result)

    # we can update the configuration to change any parameter for nested context
    with ctx.updated(
        ctx.state(OpenAIChatConfig).updated(
            model="gpt-4o",
            temperature=0.7,
        ),
    ):
        # now we are using gpt-4o with temperature of 0.7
        result = await generate_text(
            instruction="Prepare simplest completion of given text",
            input="Roses are red...",
        )

        print("RESULT GPT 4o | temperature 0.7:", result)

    # we can also update the configuration for a single call
    # here we are using gpt-3.5-turbo with temperature of 0.7
    result = await generate_text(
        instruction="Prepare simplest completion of given text",
        input="Roses are red...",
        temperature=0.7,
    )

    print("RESULT GPT 3.5 | temperature 0.7:", result)

RESULT GPT 3.5 | temperature 0.4: Violets are blue.
RESULT GPT 4o | temperature 0.7: Violets are blue,
Sugar is sweet,
And so are you.
RESULT GPT 3.5 | temperature 0.7: Violets are blue.


Since we knows the basics now we can examine the details of our execution to see what actually happened inside. We can setup the logger before execution to see context metrics logs.

In [5]:
from draive import setup_logging

setup_logging("text_completion") # setup logger

async with ctx.new(  # prepare the context and see the execution metrics report
    "text_completion",
    state=[
        LMM(invocation=openai_lmm_invocation),
        OpenAIChatConfig(  # define GPT model configuration
            model="gpt-3.5-turbo",
            temperature=0.4,
        ),
    ],
    dependencies=[OpenAIClient],
):
    await generate_text(
        instruction="Prepare simplest completion of given text",
        input="Roses are red...",
    )

    with ctx.updated(
        ctx.state(OpenAIChatConfig).updated(
            model="gpt-4o",
            temperature=0.7,
        ),
    ):
        await generate_text(
            instruction="Prepare simplest completion of given text",
            input="Roses are red...",
        )

    await generate_text(
        instruction="Prepare simplest completion of given text",
        input="Roses are red...",
        temperature=0.7,
    )


06/Jun/2024:20:57:31 +0000 [INFO] [text_completion] [369c504f84004f78b56fc33e210b366f|text_completion] started...
06/Jun/2024:20:57:31 +0000 [INFO] [text_completion] [369c504f84004f78b56fc33e210b366f|lmm_generate_text] started...
06/Jun/2024:20:57:31 +0000 [INFO] [text_completion] [369c504f84004f78b56fc33e210b366f|openai_lmm_completion] started...
06/Jun/2024:20:57:31 +0000 [DEBUG] [text_completion] [369c504f84004f78b56fc33e210b366f|openai_lmm_completion] Requested OpenAI lmm
06/Jun/2024:20:57:32 +0000 [INFO] [text_completion] [369c504f84004f78b56fc33e210b366f|openai_lmm_completion] ...finished after 0.78s
06/Jun/2024:20:57:32 +0000 [DEBUG] [text_completion] [369c504f84004f78b56fc33e210b366f|lmm_generate_text] Received text generation result
06/Jun/2024:20:57:32 +0000 [INFO] [text_completion] [369c504f84004f78b56fc33e210b366f|lmm_generate_text] ...finished after 0.78s
06/Jun/2024:20:57:32 +0000 [INFO] [text_completion] [369c504f84004f78b56fc33e210b366f|lmm_generate_text] started...
06/

The more advanced usage and use cases can be explored in other notebooks.